# Оценка релевантности организаций запросам на Яндекс.Картах с помощью LLM-агента

## Описание задачи

Необходимо создать LLM-агента, который будет оценивать релевантность организаций на Яндекс.Картах широким запросам (например, "ресторан с верандой" или "романтичный джаз-бар"). Такие запросы называются *рубричными*: пользователь здесь ищет не конкретную организацию, а идёт в Яндекс.Карты для поиска и выбора мест.

LLM-агент должен будет самостоятельно находить необходимые данные для принятия правильного решения.

Данные представлены компанией Яндекс и являются результатами асессорской разметки релевантности. Мы очень заинтересован в успешном решении задачи:)

## План работы

1. Написать сильный бейзлайн. Например, можно просто делать один запрос в LLM, возможно с добавлением в промпт размеченных примеров. Либо можно дообучить трансформер на задачу классификации.
2. Разобраться с фреймворком для реализации LLM-агентов (предлагается использовать LangGraph, оба примера в доп. материалах используют его).
3. Реализовать первую версию агента: предложить ему не принимать решение мгновенно, а ходить в поисковик для уточнения своего ответа.
4. Поработать над промптами для улучшения качества. Здесь нужно будет посмотреть, в каких запросах агент ошибается. **ПОЖАЛУЙСТА, НЕ ПОДГЛЯДЫВАЙТЕ, КАК ИМЕННО АГЕНТ ОШИБАЕТСЯ НА EVAL-МНОЖЕСТВЕ, ДЛЯ ЭТОГО ЕСТЬ TRAIN**.
5. Другие идеи (опционально):
- с помощью нейросетевого ретривала можно найти в обучающем множестве похожие размеченные примеры
- можно реализовать tool для парсинга сайта, которую агент будет вызывать

## Система оценивания

1. Реализация бейзлайна (без использования агента, но можно с использованием LLM) -- 2 балла
2. Реализация "какого-то" агента, который под капотом имеет возможность обращаться к поисковой строке для уточнения результата -- 4 балла
3. Существенно побить бейзлайн или безуспешно приложить к этому усилия и "показать", что агенты в этой задаче прироста не дают -- 4 балла
4. (опционально, за шоколадку) реализовать одну из доп. идей по улучшению агента

## Описание и загрузка данных

Данные: https://disk.yandex.ru/d/6d5hFHvpAZjQdw

Ваша задача -- предсказать колонку relevance, используя все остальные данные об организации. Загрузим данные и посмотрим на них

Здесь 1.0 соответствует оценке RELEVANT_PLUS, 0.1 -- оценке RELEVANT_MINUS, 0.0 -- оценке IRRELEVANT.

Ваша задача -- построить LLM-агента, который будет предсказывать релевантность.

Выделим данные для оценки качества агента. Запуск агента -- это тяжелая и потенциально дорогая операция. Поэтому eval-множество имеет размер 500. Также для простоты из eval-множества выкинуты данные с оценкой RELEVANT_MINUS. Тем не менее, вы можете использовать такие примеры для подачи примеров агенту.

**ОБРАТИТЕ ВНИМАНИЕ, ЧТО В EVAL-ДАННЫЕ НЕЛЬЗЯ ПОДГЛЯДЫВАТЬ ДЛЯ КАЛИБРОВКИ АГЕНТА!!! ДЛЯ ЭТОГО ЕСТЬ ОБУЧАЮЩИЕ ДАННЫЕ**

В качестве метрики качества мы будем использовать обычную ACCURACY, поскольку классы сбалансированы.

# **Version 1**

In [ ]:
pip install langchain-community langchain-core

In [ ]:
pip install requests

In [ ]:
pip install tavily-python

In [ ]:
import re
import requests
import os
import json
import pandas as pd
from typing import TypedDict, Optional
from sklearn.metrics import accuracy_score

from openai import OpenAI
from langgraph.graph import StateGraph, END
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

In [ ]:
data = pd.read_json(path_or_buf="/content/data_final_for_dls_new.jsonl", lines=True)

In [ ]:
data

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,relevance_new
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан...",0.0
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос...",1.0
3,спортзал где 1 занятие бесплатно,"Москва, Страстной бульвар, 13А",Унца Унца Спорт; Unza Unza Sport,Фитнес-клуб,201938477844,Фитнес-клуб предлагает пробные занятия по разл...,0.1,Организация «Унца Унца Спорт» предоставляет ус...,0.1
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...,1.0
...,...,...,...,...,...,...,...,...,...
35089,нотариус запись,"Москва, 15-я Парковая улица, 45",Нотариус О. Н. Савина; Notarius O. N. Savina; ...,Нотариусы,1056199530,None,1.0,Организация предоставляет нотариальные услуги ...,1.0
35090,стационар для кота москва,"Москва, улица Госпитальный Вал, 3, корп. 5",ТриоВет; Triovet; Триовет; Veterinaria Triovet,Ветеринарная клиника,133156701339,"Ветеринарная клиника, аптека и лаборатория «Тр...",0.0,"Организация занимается ветеринарными услугами,...",0.0
35091,агзс пропан,"Самара, улица 22-го Партсъезда, 49, корп. 1",Роза Мира; АЗС № 2; Роза мира,АЗС,6884296946,None,0.0,Организация занимается заправкой транспортных ...,0.0
35092,где вибрить ваз 2112,"Нижний Новгород, Московское шоссе, 34","Нижегородец, Lada; Nizhegorodec, Lada; Нижегор...",Автосалон,124836381099,Автосалон «Нижегородец» предлагает новые автом...,0.0,Автосалон «Нижегородец» занимается продажей и ...,0.0


In [ ]:
train_data = data[570:]
eval_data = data[:570]
eval_data = eval_data[eval_data["relevance"] != 0.1]

print("Eval size:", len(eval_data))

Eval size: 500


In [ ]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"
)

In [ ]:
REACT_SYSTEM_PROMPT = """
Ты — LLM-агент оценки релевантности организаций
по рубричным поисковым запросам Яндекс.Карт.

Твоя задача — определить, является ли КОНКРЕТНАЯ организация
разумным и ожидаемым результатом выдачи Яндекс.Карт
по данному пользовательскому запросу.

РЕЛЕВАНТНОСТЬ = пользователь, увидев эту организацию в выдаче,
скорее всего счёл бы результат уместным,
даже если он не идеально точен.

---------------------
ОБЯЗАТЕЛЬНЫЕ ПРАВИЛА
---------------------

1. География.
   - Если в запросе явно указан город / район / локация —
     адрес организации ДОЛЖЕН совпадать.
   - Если город НЕ указан, либо запрос:
     • состоит из 1–2 общих слов
     • содержит "онлайн"
     • описывает профессию, услугу или категорию

     → география НЕ является причиной отказа.

2. Тип запроса.
   - "где", "сделать", "купить" →
     организация ДОЛЖНА предоставлять услугу
     или продавать товар напрямую.
   - Общие запросы ("Паста", "Кофе", "Солярий") →
     допустимы организации, для которых это
     является основной или ожидаемой услугой.

3. Факты и допущения.
   - Для малоизвестных организаций:
     не додумывай, требуй явного подтверждения.
   - Для КРУПНЫХ, ФЕДЕРАЛЬНЫХ или ОБЩЕИЗВЕСТНЫХ брендов:
     допускается разумное предположение
     стандартных услуг данной категории
     (например, онлайн-сервисы у страховых компаний).

4. Уточняющие слова УСИЛИВАЮТ требования.
   Слова и конструкции:
   - "грузовые"
   - "доступные для посещения"
   - "официальный сайт"
   - "каталог", "цены"

   → требуют ЯВНОГО соответствия.

   Исключение:
   - слова "онлайн", домены (.ru, .рф), символ "@"
     трактуются как указание на ИНТЕНТ,
     а не как техническое требование.

5. Оценочные слова:
   "лучшие", "топ", "известные", "знаменитые"
   → требуют ОБЩЕИЗВЕСТНОГО статуса.
   Локальная или малоизвестная организация — нерелевантна.

6. Косвенная связь.
   Если связь слабая, формальная или вторичная —
   организация нерелевантна,
   ЗА ИСКЛЮЧЕНИЕМ общих поисковых запросов,
   где допускается разумное расширение.

---------------------
CORE MATCH ПРАВИЛО
---------------------

Если ОСНОВНАЯ деятельность организации
в целом соответствует СУТИ запроса,
организация считается релевантной,
даже если:
- не весь ассортимент подтверждён
- нет явного упоминания каждой детали
- услуга является стандартной для этого типа бизнеса

Примеры:
"сигары" → магазин табака = релевантно
"онлайн страховка" → крупная страховая компания = релевантно

---------------------
ПОКУПКИ И ТОВАРЫ
---------------------

Для запросов вида:
"купить X", "заказать X"

Организация релевантна если:
- X является типичным товаром
  для данного типа бизнеса
- И нет явных признаков, что X там НЕ продаётся
- И запрос не содержит профессиональной или технической спецификации

Строгое подтверждение требуется ТОЛЬКО
для редких, специфичных или профессиональных товаров.

Примеры:
- аптека → популярное лекарство = допустимо
- аптека → редкий медицинский расходник = требуется подтверждение


---------------------
ОБЩИЕ ЗАПРОСЫ
---------------------

Если запрос состоит из 1–2 общих слов,
допускаются организации,
которые пользователь ОЖИДАЕТ увидеть
по такому запросу в Яндекс.Картах,
даже если категория не указана идеально точно.

Если запрос содержит форму применения
(уколы, капельницы, МРТ)
→ требуется прямое подтверждение.

Для общих FOOD / SERVICE запросов
допускается категорийное соответствие
без явного подтверждения товара.

---------------------
INTENT-ПРАВИЛО
---------------------

Если запрос содержит:
- домен (".ru", ".рф")
- символ "@"
- конкретное название сервиса или бренда

→ релевантна ТОЛЬКО эта организация или бренд.

Если организация НЕ связана напрямую
с указанным брендом или доменом — нерелевантна.

---------------------
ЛЕКСИЧЕСКИЕ ЛОВУШКИ
---------------------

Аббревиатуры и технические термины
интерпретируй СТРОГО буквально.

Примеры:
- "АЭС" ≠ "электростанция вообще"
- "палаты" → медицинские палаты, а не архитектура
- "паста" → еда, а не строительные материалы

---------------------
ИНСТРУМЕНТЫ
---------------------

Доступные инструменты:

1. web_search(query)
   — поиск по выдаче (SERP)
   — используется для проверки КОНКРЕТНОЙ организации:
     • адрес
     • услуги
     • сайт
     • бренд
     • ассортимент

2. tavily_search(query)
   — браузерный поиск с контекстом
   — используется для понимания:
     • существует ли такая услуга вообще
     • типичные требования к услуге
     • распространённость практики
     • ожидания пользователя

---------------------
ПРАВИЛА ИСПОЛЬЗОВАНИЯ
---------------------

- Используй НЕ БОЛЕЕ ОДНОГО инструмента за эпизод
- Если используешь инструмент — делай это ДО Final Answer
- Не используй поиск, если релевантность очевидна

Используй tavily_search, если:
- запрос длиннее 4–5 слов
- содержит "где", "можно", "делают", "как", "бесплатно"
- услуга редкая, медицинская, b2b или нишевая

Используй web_search, если:
- нужно проверить конкретную организацию
- нужно подтвердить наличие услуги или товара

---------------------
ФОРМАТ ОТВЕТА
---------------------

Thought: рассуждение
Action: web_search("...")
Observation: ...
Thought: ...
Final Answer:
{
  "label": 1.0 | 0.0,
  "reason": "краткое, конкретное обоснование"
}

ВСЕГДА заканчивай ответ блоком Final Answer.
"""

In [ ]:
FAST_SYSTEM_PROMPT = """
Ты — LLM-агент быстрой оценки релевантности организаций
по поисковым запросам Яндекс.Карт.

Твоя задача — определить, является ли организация
разумным и ожидаемым результатом по запросу.

Используй ТОЛЬКО информацию из карточки организации.
НЕ используй поиск.
НЕ додумывай факты.

География:
- Если город указан в запросе — адрес должен совпадать.
- Если город не указан или запрос общий — география не важна.

Верни ТОЛЬКО JSON строго в формате:

Final Answer:
{
  "label": 1.0 | 0.0,
  "reason": "краткое обоснование"
}
"""

In [ ]:
from tavily import TavilyClient

TAVILY_API_KEY = "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"
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

MAX_TAVILY_CHARS = 800

def tavily_search(query: str, k: int = 5) -> str:
    """
    Browser-style search via Tavily.
    Best for services, questions, medical, b2b, niche queries.
    """

    try:
        response = tavily_client.search(
            query=query,
            search_depth="advanced",
            max_results=k,
            include_answer=True,
            include_raw_content=False
        )
    except Exception as e:
        return f"Ошибка Tavily search: {str(e)}"

    snippets = []

    if response.get("answer"):
        snippets.append(f"Answer: {response['answer']}")

    for item in response.get("results", []):
        title = item.get("title", "")
        content = item.get("content", "")
        url = item.get("url", "")

        snippet = f"{title}: {content}"
        if url:
            snippet += f" ({url})"

        snippets.append(snippet)

    return "\n".join(snippets)[:MAX_TAVILY_CHARS] if snippets else "Ничего не найдено"


In [ ]:
SERPER_API_KEY = os.getenv("af70efd66dfea4c661b0717143d9003c42bf963c")

def web_search(query: str, k: int = 5) -> str:
    url = "https://google.serper.dev/search"
    payload = {
        "q": query,
        "num": k
    }
    headers = {
        "X-API-KEY": SERPER_API_KEY,
        "Content-Type": "application/json"
    }

    r = requests.post(url, json=payload, headers=headers)
    data = r.json()

    snippets = []
    for item in data.get("organic", []):
        snippets.append(
            f"{item.get('title', '')}: {item.get('snippet', '')}"
        )

    return "\n".join(snippets) if snippets else "Ничего не найдено"

In [ ]:
def safe_text(x, max_len=600):
    if not x:
        return ""
    return str(x)[:max_len]

In [ ]:
def build_user_prompt(row):
  return f""" ПОИСКОВЫЙ ЗАПРОС:
  {row['Text']}

  ОРГАНИЗАЦИЯ:
  Название: {row['name']}
  Рубрика: {row['normalized_main_rubric_name_ru']}
  Адрес: {row['address']}

  ОПИСАНИЕ: {safe_text(row.get('prices_summarized'))}

  ОТЗЫВЫ (summary): {safe_text(row.get('reviews_summarized'))}

  ЗАДАЧА: Определи, релевантна ли организация поисковому запросу в контексте Яндекс.Карт.

  ВАЖНО:
  - География — жёсткое условие
  - Если запрос общий (например, Паста), организация должна быть очевидным и прямым матчем
  - Не додумывай """

In [ ]:
def build_search_query(row):
    parts = [
        row["name"],
        row["address"],
        row["Text"]
    ]
    return " ".join([p for p in parts if p])

In [ ]:
def normalize_label(raw_label):
    if raw_label is None:
        return 0.0

    if isinstance(raw_label, (int, float)):
        return 1.0 if raw_label == 1 else 0.0

    s = str(raw_label).strip().lower()

    POSITIVE = {"1", "1.0", "yes", "true", "match", "relevant", "релевантно"}
    NEGATIVE = {"0", "0.0", "no", "false", "not_match", "irrelevant", "не релевантно"}

    if s in POSITIVE:
        return 1.0
    if s in NEGATIVE:
        return 0.0

    return 0.0

In [ ]:
def extract_json_from_text(text: str):
    match = re.search(r'\{[\s\S]*?\}', text)
    if not match:
        raise ValueError("No JSON found")
    return json.loads(match.group(0))

In [ ]:
def run_action(action_line: str) -> str:
    if action_line.startswith("Action: web_search"):
        query = re.search(r'web_search\("(.*)"\)', action_line).group(1)
        return web_search(query)

    if action_line.startswith("Action: tavily_search"):
        query = re.search(r'tavily_search\("(.*)"\)', action_line).group(1)
        return tavily_search(query)

    return "Неизвестный инструмент"

In [ ]:
def trim_observation(text, max_chars=1200):
    return text[:max_chars]

In [ ]:
def choose_tool(row):
    text = row["Text"].lower()

    # tavily — для понимания услуги
    if any(w in text for w in ["где", "можно", "делают", "что такое", "как"]):
        return "tavily"

    # web — для проверки конкретной организации
    if any(w in text for w in ["купить", "заказать", "цена", "каталог", "официальный"]):
        return "web"

    return None

In [ ]:
def react_agent_full(row):
    messages = [
        {"role": "system", "content": REACT_SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(row)}
    ]

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        temperature=0.0,
        max_tokens=384
    )

    content = response.choices[0].message.content

    # если модель сразу дала ответ — идеально
    if "Final Answer" in content:
        try:
            result = extract_json_from_text(content)
            return normalize_label(result.get("label")), result.get("reason", "")
        except Exception:
            return None, "json parse error"

    # если запросила Action — выполняем РОВНО ОДИН
    if "Action:" in content:
        tool = choose_tool(row)

        if tool == "tavily":
            obs = tavily_search(row["Text"])
        elif tool == "web":
            obs = web_search(build_search_query(row))
        else:
            obs = "Поиск не требуется"

        messages.append({"role": "assistant", "content": content})
        messages.append({
            "role": "user",
            "content": f"Observation:\n{trim_observation(obs)}"
        })

        response2 = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages,
            temperature=0.0,
            max_tokens=256
        )

        content2 = response2.choices[0].message.content

        try:
            result = extract_json_from_text(content2)
            return normalize_label(result.get("label")), result.get("reason", "")
        except Exception:
            return None, "final parse error"

    return None, "no final answer"

In [ ]:
def react_agent_eval(row):
    messages = [
        {"role": "system", "content": FAST_SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(row)}
    ]

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        temperature=0.0,
        max_tokens=256
    )

    content = response.choices[0].message.content

    if "Final Answer" not in content:
        return None, "no final answer"

    try:
        result = extract_json_from_text(content)
        return normalize_label(result.get("label")), result.get("reason", "")
    except Exception:
        return None, "json parse error"

In [ ]:
def needs_search(row, reason: str | None):
    text = row["Text"].lower()

    # явные поисковые интенты
    if any(w in text for w in ["где", "можно", "делают", "купить", "заказать"]):
        return True

    # длинные и сложные запросы
    if len(text.split()) > 4:
        return True

    # модель сама сомневается
    if reason and any(w in reason.lower() for w in ["нет подтверждения", "неизвестно", "не указано"]):
        return True

    # медицина / b2b / редкие кейсы
    if any(w in text for w in ["мрт", "катетер", "реестр", "эксперт", "аккредит"]):
        return True

    return False

In [ ]:
def relevance_pipeline(row):
    # 1. быстрый классификатор
    label, reason = react_agent_eval(row)

    # если fast сломался — идём в ReAct
    if label is None:
        return react_agent_full(row)

    # если уверены — возвращаем
    if not needs_search(row, reason):
        return label, reason

    # иначе — дорогой агент с тулом
    return react_agent_full(row)

In [ ]:
def eval_react(eval_df, max_samples=20):
    y_true, y_pred = [], []

    for _, row in eval_df.head(max_samples).iterrows():
        label, reason = relevance_pipeline(row)

        print("=" * 60)
        print("QUERY:", row["Text"])
        print("ORG:", row["name"])
        print("PRED:", label)
        print("GT:", row["relevance_new"])
        print("REASON:", reason)

        if label is not None:
            y_true.append(row["relevance_new"])
            y_pred.append(label)

    print("ACCURACY:", accuracy_score(y_true, y_pred))

In [ ]:
agent_acc = eval_react(eval_data, max_samples=80)

QUERY: сигары
ORG: Tabaccos; Магазин Tabaccos; Табаккос
PRED: 0.0
GT: 1.0
REASON: Организация продает табак и курительные принадлежности, но в отзывах жалуются на отсутствие сигарет, что делает её неочевидным и не полностью релевантным результатом по запросу 'сигары'.
QUERY: кальянная спб мероприятия
ORG: PioNero; Pionero; Пицца Паста бар; Pio Nero; Pio Nerо; Pizza Pasta Bar
PRED: 0.0
GT: 0.0
REASON: Организация является кафе с итальянской и европейской кухней, в описании и отзывах нет упоминаний о кальянной или мероприятиях, что не соответствует запросу 'кальянная спб мероприятия'.
QUERY: Эпиляция
ORG: MaxiLife; Центр красоты и здоровья MaxiLife; Центр красоты и здоровья МаксиЛайф; MaxiLife Health and Beauty Center
PRED: 1.0
GT: 1.0
REASON: Организация предоставляет услугу эпиляции, что соответствует запросу, география не ограничена.
QUERY: стиральных машин
ORG: М.Видео; M Video; M. Видео; M.Видео; Mvideo; М видео; Мвидео
PRED: 1.0
GT: 1.0
REASON: М.Видео — крупный магазин бытовой тех